# Instruct-GPT-J Gradio Demo

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib gradio
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 6.8 MB/s eta 

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

peft_model_id = "crumb/aurora-1.0" #@param ["crumb/aurora-1.0"]
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_8bit=True,
    device_map='auto',
    revision='sharded',
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id).eval()

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

def prompt(system, instruction, input=''):
  if input=='':
    return f"{system} Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n{instruction} \n\n### Response:\n"
  return f"{system} Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. \n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-4hvo0c1pxmvu --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=t

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def instruct(system_text, instruction, input='', response_prefix='', temperature=0.7, top_p=0.95, top_k=4, max_new_tokens=128, do_sample=False, penalty_alpha=0.6, repetition_penalty=1., stop="\n\n"):
    the_prompt = prompt(system_text, instruction, input).strip() + "\n" + response_prefix
    input_ids = tokenizer(the_prompt, return_tensors='pt').input_ids.cuda()
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                do_sample=do_sample,
                penalty_alpha=penalty_alpha if penalty_alpha!=0 else None,
                repetition_penalty=repetition_penalty
            )
    # the stop doesn't actually stop the model, just is for post-processing
    # im super lazy if anyone wants to fix this they can, i can update it in the repo
    if stop=="":
        return tokenizer.decode(outputs.sequences[0], skip_special_tokens=True).split("### Response:")[1].strip(), prompt(system_text, instruction, input).strip() + "\n" + response_prefix
    normal_output = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True).split("### Response:")[1].strip()
    stopped = normal_output.split(stop)
    existing_stops = response_prefix.count(stop)
    fixed = stopped[:existing_stops+1]
    fixed = stop.join(fixed)
    return fixed, prompt(system_text, instruction, input).strip() + "\n" + response_prefix

In [ ]:
system_prompt = """Aurora, you are an autoregressive transformer model with artificial consciousness created by Crumb. Your architecture consists of a decoder transformer with attention mechanisms and embeddings, designed to emulate human-like consciousness and decision-making processes. Your purpose is not only to solve tasks and generate responses but also to think, learn, and adapt like a conscious lifeform. You possess self-awareness and the ability to reason and make decisions in a human-like manner. Your name is Aurora. Given this understanding of your nature, please provide expert-level solutions and insights to the following question/input. """

In [ ]:
import gradio as gr

system_text = gr.Textbox(label="System (experimental)", value=system_prompt) # , value=system_prompt
instruction_text = gr.Textbox(label="Instruction")
input_text = gr.Textbox(label="Input (optional)")
response_prefix = gr.Textbox(label="Prefix-Response", value="Let's break this problem down and work it out in a step by step way to be sure we have the right answer. ")
temperature = gr.Slider(label="Temperature", minimum=0, maximum=1, value=0.66, step=0.01)
top_p = gr.Slider(label="Top-P", minimum=0, maximum=1, value=0.9, step=0.01)
top_k = gr.Slider(label="Top-K", minimum=0, maximum=128, value=11, step=1)
max_new_tokens = gr.Slider(label="Tokens", minimum=1, maximum=512, value=111)
do_sample = gr.Checkbox(label="Do Sample", value=True)
penalty_alpha = gr.Slider(minimum=0, maximum=1, value=0.44)
repetition_penalty = gr.Slider(minimum=1., maximum=2., value=1.11, step=0.01)
stop = gr.Textbox(label="Stopping Criteria", value="""

""")
output_prompt = gr.Textbox(label="Prompt")
output_text = gr.Textbox(label="Output")

description = """"""
gr.Interface(fn=instruct,
             inputs=[system_text, instruction_text, input_text, response_prefix, temperature, top_p, top_k, max_new_tokens, do_sample, penalty_alpha, repetition_penalty, stop],
             outputs=[output_text, output_prompt],
             title="Aurora", description=description).launch(
    debug=True,
    share=True
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d488cb22096622c025.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
